In [1]:
library(tidyverse)
library(inspectdf)
library(readxl)
library(tidylog)

── Attaching core tidyverse packages ───────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ─────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘tidylog’


The following objects are masked from ‘package:dplyr’:

    add_count, add_tally, anti_join, count, distinct, distinct_all,
    distinct_at, distinct_if, filter, filter_all, filter_at, filter_if,
    full_join, group_by, group_by_all, group_by_at, group_by_if,
    inner_join, left_join, mutate, mutate_all, mutate_at, mutate_if,
    relocate, rename, rename_all, rename_at, rename_if, rename_with,
   

In [2]:
key_cols <- c("data_source", "study_id", "site_id", "core_id", "method_id", "depth_min", "depth_max",
              "sample_id", "dry_bulk_density", "fraction_organic_matter", "year", "month", 
              "latitude", "longitude", "species_code","code_type")

In [3]:
(dir_list <- list.dirs("../data/ZAF", recursive = TRUE, full.names = TRUE)[-1])

[1] "../data/ZAF/Bossert_et_al_2024" "../data/ZAF/Bossert_et_al_2025"
[3] "../data/ZAF/Machite_et_al"      "../data/ZAF/Ndhlovu_et_al"

In [4]:
read_file_smart <- function(file_path) {
    
  message(file_path)
  suppressMessages(suppressWarnings(
  if (str_detect(file_path, "\\.csv$")) {
    return(read_csv(file_path))
  } else if (str_detect(file_path, "\\.xlsx?$")) {
    return(read_excel(file_path))
  } else {
    stop("Unsupported file type: ", file_path)
  }))
}

In [8]:
extract_data <- function(search_path){

                message(search_path)
                (file_list <- list.files(search_path, full.names = TRUE))
                depthseries <- read_file_smart(file_list[str_detect(file_list, "depthseries")])
                print(dim(depthseries))
                message(colnames(depthseries))
                cores <- read_file_smart(file_list[str_detect(file_list, "cores")])
                species <- read_file_smart(file_list[str_detect(file_list, "species")])
                ccn_db <- depthseries |>
                          dplyr::left_join(cores) |>
                          dplyr::left_join(species) |>
                          dplyr::mutate(data_source = basename(search_path),
                                        fraction_organic_matter = if_else(data_source == "Machite_et_al", fraction_organic_matter * 100, fraction_organic_matter)) |>
                          dplyr::relocate(data_source) |>
                          dplyr::select(any_of(key_cols))
    
                print(dim(ccn_db))

  return(ccn_db)
}

In [9]:
dir_list

[1] "../data/ZAF/Bossert_et_al_2024" "../data/ZAF/Bossert_et_al_2025"
[3] "../data/ZAF/Machite_et_al"      "../data/ZAF/Ndhlovu_et_al"

In [10]:
zaf_data_ls <- lapply(dir_list, extract_data)

../data/ZAF/Bossert_et_al_2024

../data/ZAF/Bossert_et_al_2024/Bossert_et_al_2024_depthseries.xlsx



[1] 180   9


study_idsite_idcore_idmethod_iddepth_mindepth_maxsample_iddry_bulk_densityfraction_organic_matter

../data/ZAF/Bossert_et_al_2024/Bossert_et_al_2024_cores.xlsx

../data/ZAF/Bossert_et_al_2024/Bossert_et_al_2024_species.xlsx

Joining with `by = join_by(study_id, site_id, core_id)`
Joining with `by = join_by(study_id, site_id, core_id, habitat)`


[1] 180  16


../data/ZAF/Bossert_et_al_2025

../data/ZAF/Bossert_et_al_2025/Bossert_et_al_2025_depthseries_Langebaan.xlsx



...

[1] 3480   13


../data/ZAF/Ndhlovu_et_al

../data/ZAF/Ndhlovu_et_al/Ndhlovu_et_al_2025_depthseries.csv



[1] 226  10


study_idsite_idplot_idcore_idmethod_iddepth_mindepth_maxsample_iddry_bulk_densityfraction_organic_matter

../data/ZAF/Ndhlovu_et_al/Ndhlovu_et_al_2025_cores.csv

../data/ZAF/Ndhlovu_et_al/Ndhlovu_et_al_2025_species.csv

Joining with `by = join_by(study_id, site_id, plot_id, core_id)`
Joining with `by = join_by(study_id, site_id, plot_id, core_id, habitat)`


[1] 226  16


In [11]:
(shared_cols <- Reduce(intersect, lapply(zaf_data_ls, names)))

[1] "data_source"             "study_id"               
 [3] "site_id"                 "core_id"                
 [5] "depth_min"               "depth_max"              
 [7] "dry_bulk_density"        "fraction_organic_matter"
 [9] "year"                    "latitude"               
[11] "longitude"               "species_code"           
[13] "code_type"

In [12]:
(lapply(zaf_data_ls, dim)) |> data.frame() |> rowSums()

[1] 4241   61

In [13]:
zaf_data_common <- lapply(zaf_data_ls, function(df) df[, shared_cols, drop = FALSE])
zaf_data_df <- do.call(rbind, zaf_data_common)

In [14]:
unique(zaf_data_df$study_id)

[1] "Bossert_et_al_2024"      "Bossert_et_al_2025"     
 [3] NA                        "Adams_and_Human_2016"   
 [5] "Bekker_2015"             "Bezuidenhout_et_al_2011"
 [7] "Brown_and_Rajkaran_2020" "Els_2017"               
 [9] "Els_2019"                "Geldenhyus_et_al_2016"  
[11] "Hoppe-Speer_et_al_2013"  "Human_et_al_2022"       
[13] "Johnson_et_al_2020"      "Lemley_2018"            
[15] "Matabane_2018"           "Mbense_et_al_2016"      
[17] "Mbense_2019"             "Naidoo_2014"            
[19] "Peer_et_al_2018"         "Rajkaran_and_Adams_2011"
[21] "Rautenbach_2015"         "Raw_et_al_2019"         
[23] "Veldkornet_2016_PhD"     "Veldkornet_et_al_2016"  
[25] "Verle_2013"              "Vromans_2010"           
[27] "Wooldridge_et_al_2016"   "Ndhlovu_et_al_2024"

In [15]:
unique(zaf_data_df$data_source)

[1] "Bossert_et_al_2024" "Bossert_et_al_2025" "Machite_et_al"     
[4] "Ndhlovu_et_al"

In [16]:
(Zcapensis_zaf_raw <- zaf_data_df |>
                      dplyr::filter(species_code == "Zostera capensis")) |> dim()

[1] 573  13

In [17]:
dim(Zcapensis_zaf_raw)

[1] 573  13

In [18]:
dplyr::filter(Zcapensis_zaf_raw, is.na(dry_bulk_density)) |> dim()

[1] 167  13

In [19]:
dplyr::filter(Zcapensis_zaf_raw, is.na(fraction_organic_matter)) |> dim()

[1] 19 13

In [20]:
dplyr::filter(Zcapensis_zaf_raw, is.na(depth_min) | is.na(depth_max)) |> dim()

[1]  0 13

In [21]:
colnames(Zcapensis_zaf_raw)

[1] "data_source"             "study_id"               
 [3] "site_id"                 "core_id"                
 [5] "depth_min"               "depth_max"              
 [7] "dry_bulk_density"        "fraction_organic_matter"
 [9] "year"                    "latitude"               
[11] "longitude"               "species_code"           
[13] "code_type"

In [22]:
(Zcapensis_zaf <- dplyr::filter(Zcapensis_zaf_raw, !is.na(depth_min) | !is.na(depth_max))) |> dim()

[1] 573  13

In [23]:
write_rds(Zcapensis_zaf, "../data/zaf_data_df.rds")

In [24]:
xtabs(~study_id, data = Zcapensis_zaf)

study_id
            Bekker_2015 Bezuidenhout_et_al_2011      Bossert_et_al_2024 
                      4                      24                     180 
Brown_and_Rajkaran_2020                Els_2017                Els_2019 
                      7                      56                      48 
            Lemley_2018      Ndhlovu_et_al_2024 Rajkaran_and_Adams_2011 
                      5                     226                      13 
    Veldkornet_2016_PhD 
                     10 

In [25]:
(Zcapensis_cores <- Zcapensis_zaf |>
                    dplyr::group_by(data_source, study_id, site_id, core_id) |>
                    dplyr::summarise(N_samples = n(), max_depth = max(depth_max))) |> dim()

`summarise()` has grouped output by 'data_source', 'study_id', 'site_id'. You can override using the
`.groups` argument.


[1] 137   6

In [26]:
colnames(Zcapensis_zaf)

[1] "data_source"             "study_id"               
 [3] "site_id"                 "core_id"                
 [5] "depth_min"               "depth_max"              
 [7] "dry_bulk_density"        "fraction_organic_matter"
 [9] "year"                    "latitude"               
[11] "longitude"               "species_code"           
[13] "code_type"

In [27]:
xtabs(~study_id, data = Zcapensis_cores)

study_id
            Bekker_2015 Bezuidenhout_et_al_2011      Bossert_et_al_2024 
                      4                       8                      45 
Brown_and_Rajkaran_2020                Els_2017                Els_2019 
                      1                       4                      12 
            Lemley_2018      Ndhlovu_et_al_2024 Rajkaran_and_Adams_2011 
                      5                      47                       1 
    Veldkornet_2016_PhD 
                     10 

In [28]:
Zcapensis_cores |>
dplyr::group_by(data_source, study_id,	site_id, max_depth) |>
dplyr::summarise(count = n()) |>
dplyr::ungroup() |>
dplyr::arrange(study_id, site_id) 

`summarise()` has grouped output by 'data_source', 'study_id', 'site_id'. You can override using the
`.groups` argument.


data_source,study_id,site_id,max_depth,count
<chr>,<chr>,<chr>,<dbl>,<int>
Machite_et_al,Bekker_2015,Knysna,5,4
Machite_et_al,Bezuidenhout_et_al_2011,Kromme,120,8
Bossert_et_al_2024,Bossert_et_al_2024,Berg,15,30
Bossert_et_al_2024,Bossert_et_al_2024,Breede,15,15
Machite_et_al,Brown_and_Rajkaran_2020,Kobonqaba,21,1
Machite_et_al,Els_2017,Knysna,110,4
Machite_et_al,Els_2019,Swartkops,100,12
Machite_et_al,Lemley_2018,Kariega,5,5
Ndhlovu_et_al,Ndhlovu_et_al_2024,Berg,50,8


In [33]:
write.csv(Zcapensis_zaf, "../data/Zcapensis_Cstocks.csv", row.names = FALSE, quote = FALSE)